# VS Code Jupyter Connection Setup

## To connect VS Code to this Docker Jupyter server:

1. **Install Extensions**: Make sure you have "Jupyter" and "Python" extensions installed in VS Code
2. **Open Command Palette**: Press `Ctrl+Shift+P` (Windows) or `Cmd+Shift+P` (Mac)
3. **Select Jupyter Server**: Type "Jupyter: Select Interpreter to Start Jupyter Server"
4. **Choose Existing Server**: Select "Existing" when prompted
5. **Enter Server URL**: Use `http://localhost:8888` (or `http://localhost:8080` if that's where it's running)
6. **No Password**: Leave password field empty (configured with no authentication)

**Alternative Method:**
- Click on the kernel picker in the top-right of any `.ipynb` file
- Select "Select Another Kernel..."
- Choose "Existing Jupyter Server"
- Enter: `http://localhost:8888`

# PySpark Tutorial - SQL Server to Parquet

This notebook demonstrates how to extract data from SQL Server and save it as Parquet files using PySpark in a Docker environment.

## 1. Import Required Libraries

First, let's import the necessary libraries for our data extraction process.

In [ ]:
import os
import sys
from dotenv import load_dotenv
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from datetime import datetime

# Load environment variables
load_dotenv()

print("Libraries imported successfully!")

Libraries imported successfully!


In [6]:
env = load_dotenv()


True


## 2. Create Spark Session

Let's create a Spark session optimized for our Docker environment.

In [7]:
# Create Spark session with SQL Server JDBC driver
spark = SparkSession.builder \
    .appName("NotebookSQLToParquet") \
    .master("local[*]") \
    .config("spark.sql.adaptive.enabled", "true") \
    .config("spark.sql.adaptive.coalescePartitions.enabled", "true") \
    .config("spark.jars.packages", "com.microsoft.sqlserver:mssql-jdbc:12.2.0.jre8") \
    .getOrCreate()

print(f"Spark version: {spark.version}")
print(f"Spark UI available at: http://localhost:4040")

:: loading settings :: url = jar:file:/opt/spark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
com.microsoft.sqlserver#mssql-jdbc added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-44d3a2b0-dd3e-4ba2-9ecd-2c3f9ec3a5b1;1.0
	confs: [default]
	found com.microsoft.sqlserver#mssql-jdbc;12.2.0.jre8 in central
downloading https://repo1.maven.org/maven2/com/microsoft/sqlserver/mssql-jdbc/12.2.0.jre8/mssql-jdbc-12.2.0.jre8.jar ...
	[SUCCESSFUL ] com.microsoft.sqlserver#mssql-jdbc;12.2.0.jre8!mssql-jdbc.jar (389ms)
:: resolution report :: resolve 578ms :: artifacts dl 391ms
	:: modules in use:
	com.microsoft.sqlserver#mssql-jdbc;12.2.0.jre8 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      def

Spark version: 3.5.0
Spark UI available at: http://localhost:4040


## 3. Test Spark with Sample Data

Let's create a simple DataFrame to verify Spark is working correctly.

In [ ]:
# Create a test DataFrame
test_data = [("Alice", 25), ("Bob", 30), ("Charlie", 35)]
columns = ["Name", "Age"]

test_df = spark.createDataFrame(test_data, columns)
test_df.show()

print(f"DataFrame has {test_df.count()} rows and {len(test_df.columns)} columns")

## 4. Connect to SQL Server

Now let's connect to your SQL Server and extract some real data.

In [ ]:
# Get connection details from environment variables
server = os.getenv("SQL_SERVER", "192.168.56.1")
database = os.getenv("SQL_DATABASE", "AdventureWorks2022")
username = os.getenv("SQL_USERNAME", "airbyte_user")
password = os.getenv("SQL_PASSWORD", "airbyte_user")

# Create JDBC URL
jdbc_url = f"jdbc:sqlserver://{server};databaseName={database};trustServerCertificate=true"

# Connection properties
connection_properties = {
    "driver": "com.microsoft.sqlserver.jdbc.SQLServerDriver",
    "user": username,
    "password": password
}

print(f"Connecting to: {server}/{database}")
print(f"JDBC URL: {jdbc_url}")

## 5. Extract Data from SQL Server

Let's extract some data from the Employee table.

In [ ]:
# Extract data from Employee table
table_name = "[HumanResources].[Employee]"

try:
    df = spark.read.jdbc(
        url=jdbc_url,
        table=table_name,
        properties=connection_properties
    )
    
    print(f"Successfully connected to SQL Server!")
    print(f"Table: {table_name}")
    print(f"Rows: {df.count()}")
    print(f"Columns: {len(df.columns)}")
    
    # Show schema
    print("\nSchema:")
    df.printSchema()
    
except Exception as e:
    print(f"Error connecting to SQL Server: {e}")

## 6. Explore the Data

Let's take a look at the first few rows of our data.

In [ ]:
# Show first 10 rows
df.show(10)

# Show specific columns
df.select("BusinessEntityID", "JobTitle", "BirthDate", "HireDate").show(10)

## 7. Data Analysis with Spark SQL

Let's perform some analysis using Spark SQL.

In [ ]:
# Create a temporary view for SQL queries
df.createOrReplaceTempView("employees")

# Count employees by job title
job_counts = spark.sql("""
    SELECT JobTitle, COUNT(*) as employee_count
    FROM employees 
    GROUP BY JobTitle 
    ORDER BY employee_count DESC
""")

print("Employee counts by job title:")
job_counts.show()

## 8. Save Data as Parquet

Finally, let's save our data as a Parquet file with compression.

In [ ]:
# Add extraction timestamp
df_with_timestamp = df.withColumn("extraction_timestamp", 
                                 lit(datetime.now().strftime("%Y-%m-%d %H:%M:%S")))

# Save as Parquet with Snappy compression
output_path = "/data/output/employee_notebook.parquet"

df_with_timestamp.write \
    .option("compression", "snappy") \
    .mode("overwrite") \
    .parquet(output_path)

print(f"Data saved to: {output_path}")
print(f"File will be available at: ./data/output/employee_notebook.parquet")

## 9. Verify Saved Data

Let's read back the Parquet file to verify it was saved correctly.

In [ ]:
# Read the Parquet file back
saved_df = spark.read.parquet(output_path)

print(f"Parquet file contains {saved_df.count()} rows")
print("Sample data from Parquet file:")
saved_df.select("BusinessEntityID", "JobTitle", "extraction_timestamp").show(5)

## 10. Cleanup

Don't forget to stop the Spark session when you're done.

In [ ]:
# Stop Spark session
# spark.stop()
# print("Spark session stopped")

# Uncomment the lines above when you're completely done
print("Notebook completed successfully!")
print("\nNext steps:")
print("1. Check ./data/output/ for your Parquet files")
print("2. Visit http://localhost:4040 for Spark UI")
print("3. Experiment with different SQL queries and transformations")